In [230]:
from pyspark.sql.functions import udf, collect_list
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType


off_train = spark.read.csv('ccf_offline_stage1_train.csv', header=True, inferSchema=True)
off_test = spark.read.csv('ccf_offline_stage1_test_revised.csv', header = True, inferSchema=True)
on_train = spark.read.csv('ccf_online_stage1_train.csv', header = True, inferSchema = True)

In [231]:
dataset3 = off_test
feature3 = off_train.filter(((off_train.Date>='20160315')&(off_train.Date<='20160630'))\
                           |((off_train.Date=='null') & (off_train.Date_received>='20160315')&(off_train.Date_received<='20160630')))\
                        .select('*')
dataset2 = off_train.filter((off_train.Date_received>='20160515')&(off_train.Date_received<='20160615')).select('*')
feature2 = off_train.filter((off_train.Date>='20160201')&(off_train.Date<='20160514')|((off_train.Date=='null')&(off_train.Date_received>='20160201')&(off_train.Date_received<='20160514'))).select('*')
dataset1 = off_train.filter((off_train.Date_received>='20160414')&(off_train.Date_received<='20160514')).select('*')
feature1 = off_train.filter((off_train.Date>='20160101')&(off_train.Date<='20160413')|((off_train.Date=='null')&(off_train.Date_received>='20160101')&(off_train.Date_received<='20160413'))).select('*')

In [234]:
###5. other feature:###

#for dataset3
t = dataset3.select('User_id')
t = t.select('User_id').groupBy('User_id').count().withColumnRenamed('count', 'Receive_all_coupon_count')

#print(t.orderBy('User_id').show())

In [235]:
t1 = dataset3.select('User_id','Coupon_id')
t1 = t1.select('User_id','Coupon_id').groupBy('User_id','Coupon_id').count().withColumnRenamed('Count', 'Receive_same_coupon_count')

#print(t1.orderBy('User_id').show())

In [236]:
t2 = dataset3.select('User_id', 'Coupon_id', 'Date_received')

In [237]:
from pyspark.sql import functions as F
t2 = t2.groupby(['User_id', 'Coupon_id']).agg(F.max("Date_received"),F.min("Date_received"))
t2 = t2.withColumnRenamed('max(Date_received)','Max_date_received').withColumnRenamed('Min(Date_received)','Min_date_received')
#t2.show()

In [240]:
t3 = dataset3.select('User_id','Coupon_id','Date_received')
t3 = t3.join(t2, ['User_id','Coupon_id'])
t3 = t3.withColumn('Month_same_coupon_lastone', col('Max_date_received') - col('date_received'))
t3 = t3.withColumn('Month_same_coupon_firstone',col('date_received') - col('Min_date_received'))                   
t3.show()

+-------+---------+-------------+-----------------+-----------------+-------------------------+--------------------------+
|User_id|Coupon_id|Date_received|Max_date_received|Min_date_received|Month_same_coupon_lastone|Month_same_coupon_firstone|
+-------+---------+-------------+-----------------+-----------------+-------------------------+--------------------------+
|3649795|    10418|     20160701|         20160701|         20160701|                        0|                         0|
|3025159|    10438|     20160701|         20160701|         20160701|                        0|                         0|
|2296075|    13602|     20160728|         20160728|         20160728|                        0|                         0|
|7346734|     4405|     20160701|         20160701|         20160701|                        0|                         0|
|2970925|     5178|     20160722|         20160728|         20160711|                        6|                        11|
|2970925|     51

In [239]:
def is_firstlastone(x):
    if x==0:
        return 1
    elif x>0:
        return 0
    else:
        return -1 #those only receive once

In [214]:
t3.withColumn('Month_same_coupon_lastone', is_firstlastone(col('Month_same_coupon_lastone')))
#t3 = t3.withColumn('Month_same_coupon_firstone', is_firstlastone(col'Month_same_coupon_firstone'))
t3 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone','Month_same_coupon_firstone')

ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

In [215]:
t4 = dataset3.select('User_id','Date_received')
t4 = t4.select('Date_received','User_id',).groupBy('User_id','Date_received').count().withColumnRenamed('count', 'Receive_all_coupon_count')

#print(t4.orderBy('user_id').show())

In [216]:
t5 = dataset3.select('User_id','Coupon_id','Date_received')
t5 = t5.select('User_id','Coupon_id','Date_received').groupBy('User_id','Coupon_id','Date_received').count().withColumnRenamed('count', 'Receive_same_coupon_count')

#print(t5.orderBy('user_id').show())

In [217]:
t6 = dataset3.select('User_id','Coupon_id','Date_received')
t6 = t6.withColumn('Date_received',(col('Date_received').astype('string')))

In [218]:
link = udf(lambda x: ':'.join(x))
t6 = t6.groupby('User_id', 'Coupon_id').agg(collect_list('Date_received').alias('Date_received')).withColumn('Date_received', link('Date_received'))
t6 = t6.withColumnRenamed('Date_received', 'Dates')

#t6.show()

In [219]:
def get_day_gap_before(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))-date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
        
def get_day_gap_after(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)

In [227]:
t7 = dataset3.select('User_id','Coupon_id','Date_received')
t7 = t7.join(t6,['User_id','Coupon_id'])
#t7 = t7.withColumn('Date_received_date',((col('Date_received').astype('string')) + '-' + t7.dates))
#t7.withColumn('Day_gap_before', get_day_gap_before(col('date_received_date')))
#t7.withColumn('Day_gap_after', get_day_gap_after(col('date_received_date')))
#t7 = t7.select('User_id','Coupon_id','Date_received','Day_gap_before','Day_gap_after')

#t7.show()

In [228]:
other_feature3 = t1.join(t, 'User_id')
other_feature3 = other_feature3.join(t3,['user_id','coupon_id'])
other_feature3 = other_feature3.join(t4, ['User_id','Date_received'])
other_feature3 = other_feature3.join(t5, ['User_id','Coupon_id','Date_received'])
other_feature3 = other_feature3.join(t7, ['User_id','Coupon_id','Date_received'])
#other_feature3.write.csv('coupon3_featureother_feature3', header = True)

other_feature3.show()

+-------+---------+-------------+-------------------------+------------------------+-----------------+-----------------+-------------------------+--------------------------+------------------------+-------------------------+--------------------+
|User_id|Coupon_id|Date_received|Receive_same_coupon_count|Receive_all_coupon_count|Max_date_received|Min_date_received|Month_same_coupon_lastone|Month_same_coupon_firstone|Receive_all_coupon_count|Receive_same_coupon_count|               Dates|
+-------+---------+-------------+-------------------------+------------------------+-----------------+-----------------+-------------------------+--------------------------+------------------------+-------------------------+--------------------+
|  11170|    11873|     20160730|                        4|                       6|         20160730|         20160715|                        0|                        15|                       1|                        1|20160717:20160715...|
|  28927|     99

In [ ]:
#for dataset2

In [ ]:
t = dataset2.select('User_id')
t = t.select('User_id').groupBy('User_id').count().withColumnRenamed('count', 'Receive_all_coupon_count')

#print(t.orderBy('User_id').show())

In [ ]:
t1 = dataset2.select('User_id','Coupon_id')
t1 = t1.select('User_id','Coupon_id').groupBy('User_id','Coupon_id').count().withColumnRenamed('Count', 'Receive_same_coupon_count')

#print(t1.orderBy('User_id').show())

In [ ]:
t2 = dataset2.select('User_id', 'Coupon_id', 'Date_received')

In [ ]:
from pyspark.sql import functions as F
t2 = t2.groupby(['User_id', 'Coupon_id']).agg(F.max("Date_received"),F.min("Date_received"))
t2 = t2.withColumnRenamed('max(Date_received)','Max_date_received').withColumnRenamed('Min(Date_received)','Min_date_received')
#t2.show()

In [ ]:
t3 = dataset2.select('User_id','Coupon_id','Date_received')
t3 = t3.join(t2, ['User_id','Coupon_id'])
t3 = t3.withColumn('Month_same_coupon_lastone', col('Max_date_received') - col('date_received'))
t3 = t3.withColumn('Month_same_coupon_firstone',col('date_received') - col('Min_date_received'))                   
#t3.show()

In [ ]:
def is_firstlastone(x):
    if x==0:
        return 1
    elif x>0:
        return 0
    else:
        return -1 #those only receive once

In [ ]:
t3.withColumn('Month_same_coupon_lastone', is_firstlastone(col('Month_same_coupon_lastone')))
#t3 = t3.withColumn('Month_same_coupon_firstone', is_firstlastone(col'Month_same_coupon_firstone'))
t3 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone','Month_same_coupon_firstone')

In [ ]:
t4 = dataset2.select('User_id','Date_received')
t4 = t4.select('Date_received','User_id',).groupBy('User_id','Date_received').count().withColumnRenamed('count', 'Receive_all_coupon_count')

#print(t4.orderBy('user_id').show())

In [ ]:
t5 = dataset2.select('User_id','Coupon_id','Date_received')
t5 = t5.select('User_id','Coupon_id','Date_received').groupBy('User_id','Coupon_id','Date_received').count().withColumnRenamed('count', 'Receive_same_coupon_count')

#print(t5.orderBy('user_id').show())

In [ ]:
t6 = dataset2.select('User_id','Coupon_id','Date_received')
t6 = t6.withColumn('Date_received',(col('Date_received').astype('string')))

In [ ]:
link = udf(lambda x: ':'.join(x))
t6 = t6.groupby('User_id', 'Coupon_id').agg(collect_list('Date_received').alias('Date_received')).withColumn('Date_received', link('Date_received'))
t6 = t6.withColumnRenamed('Date_received', 'Dates')

#t6.show()

In [ ]:
def get_day_gap_before(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))-date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
        
def get_day_gap_after(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)

In [ ]:
t7 = dataset2.select('User_id','Coupon_id','Date_received')
t7 = t7.join(t6,['User_id','Coupon_id'])
#t7 = t7.withColumn('Date_received_date',((col('Date_received').astype('string')) + '-' + t7.dates))
#t7.withColumn('Day_gap_before', get_day_gap_before(col('date_received_date')))
#t7.withColumn('Day_gap_after', get_day_gap_after(col('date_received_date')))
#t7 = t7.select('User_id','Coupon_id','Date_received','Day_gap_before','Day_gap_after')

#t7.show()

In [ ]:
other_feature2 = t1.join(t, 'User_id')
other_feature2 = other_feature3.join(t3,['user_id','coupon_id'])
other_feature2 = other_feature3.join(t4, ['User_id','Date_received'])
other_feature2 = other_feature3.join(t5, ['User_id','Coupon_id','Date_received'])
other_feature2 = other_feature3.join(t7, ['User_id','Coupon_id','Date_received'])
#other_feature2.write.csv('coupon3_featureother_feature3', header = True)

other_feature2.show()

In [ ]:
#for dataset1

In [ ]:
t = dataset1.select('User_id')
t = t.select('User_id').groupBy('User_id').count().withColumnRenamed('count', 'Receive_all_coupon_count')

#print(t.orderBy('User_id').show())

In [ ]:
t1 = dataset1.select('User_id','Coupon_id')
t1 = t1.select('User_id','Coupon_id').groupBy('User_id','Coupon_id').count().withColumnRenamed('Count', 'Receive_same_coupon_count')

#print(t1.orderBy('User_id').show())

In [ ]:
t2 = dataset1.select('User_id', 'Coupon_id', 'Date_received')

In [ ]:
from pyspark.sql import functions as F
t2 = t2.groupby(['User_id', 'Coupon_id']).agg(F.max("Date_received"),F.min("Date_received"))
t2 = t2.withColumnRenamed('max(Date_received)','Max_date_received').withColumnRenamed('Min(Date_received)','Min_date_received')
#t2.show()

In [ ]:
t3 = dataset1.select('User_id','Coupon_id','Date_received')
t3 = t3.join(t2, ['User_id','Coupon_id'])
t3 = t3.withColumn('Month_same_coupon_lastone', col('Max_date_received') - col('date_received'))
t3 = t3.withColumn('Month_same_coupon_firstone',col('date_received') - col('Min_date_received'))                   
#t3.show()

In [ ]:
def is_firstlastone(x):
    if x==0:
        return 1
    elif x>0:
        return 0
    else:
        return -1 #those only receive once

In [ ]:
t3.withColumn('Month_same_coupon_lastone', is_firstlastone(col('Month_same_coupon_lastone')))
#t3 = t3.withColumn('Month_same_coupon_firstone', is_firstlastone(col'Month_same_coupon_firstone'))
t3 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone','Month_same_coupon_firstone')

In [ ]:
t4 = dataset1.select('User_id','Date_received')
t4 = t4.select('Date_received','User_id',).groupBy('User_id','Date_received').count().withColumnRenamed('count', 'Receive_all_coupon_count')

#print(t4.orderBy('user_id').show())

In [ ]:
t5 = dataset1.select('User_id','Coupon_id','Date_received')
t5 = t5.select('User_id','Coupon_id','Date_received').groupBy('User_id','Coupon_id','Date_received').count().withColumnRenamed('count', 'Receive_same_coupon_count')

#print(t5.orderBy('user_id').show())

In [ ]:
t6 = dataset1.select('User_id','Coupon_id','Date_received')
t6 = t6.withColumn('Date_received',(col('Date_received').astype('string')))

In [ ]:
link = udf(lambda x: ':'.join(x))
t6 = t6.groupby('User_id', 'Coupon_id').agg(collect_list('Date_received').alias('Date_received')).withColumn('Date_received', link('Date_received'))
t6 = t6.withColumnRenamed('Date_received', 'Dates')

#t6.show()

In [ ]:
def get_day_gap_before(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))-date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
        
def get_day_gap_after(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)

In [ ]:
t7 = dataset1.select('User_id','Coupon_id','Date_received')
t7 = t7.join(t6,['User_id','Coupon_id'])
#t7 = t7.withColumn('Date_received_date',((col('Date_received').astype('string')) + '-' + t7.dates))
#t7.withColumn('Day_gap_before', get_day_gap_before(col('date_received_date')))
#t7.withColumn('Day_gap_after', get_day_gap_after(col('date_received_date')))
#t7 = t7.select('User_id','Coupon_id','Date_received','Day_gap_before','Day_gap_after')

#t7.show()

In [ ]:
other_feature1 = t1.join(t, 'User_id')
other_feature1 = other_feature3.join(t3,['user_id','coupon_id'])
other_feature1 = other_feature3.join(t4, ['User_id','Date_received'])
other_feature1 = other_feature3.join(t5, ['User_id','Coupon_id','Date_received'])
other_feature1 = other_feature3.join(t7, ['User_id','Coupon_id','Date_received'])
#other_feature1.write.csv('coupon3_featureother_feature3', header = True)

other_feature1.show()